In [86]:
import pandas
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp
import pickle

In [5]:
# Load the data pickle
df = pandas.read_pickle('cleaned_articles.pkl')

In [6]:
# Prepare the data for Doc2Vec
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['tokens'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=24)

# Define a function to vectorize a document
def vectorize_doc(row):
    print(f'{row.name}\r', end='')
    doc = row['tokens']
    return model.infer_vector(doc)

# Apply the function to each row in the DataFrame
df['doc2vec'] = df.apply(vectorize_doc, axis=1)

In [99]:
# Encode the authors and publications with onehot encoding

publication_encoder = OneHotEncoder()
publications = publication_encoder.fit_transform(df['publication'].values.reshape(-1,1))
with open('publication_encoder.pkl', 'wb') as f:
    pickle.dump(publication_encoder, f)

author_encoder = OneHotEncoder()
authors = author_encoder.fit_transform(df['author'].values.reshape(-1,1))
with open('author_encoder.pkl', 'wb') as f:
    pickle.dump(author_encoder, f)

In [100]:
# Vectorize the documents using TFIDF
df['joined_tokens'] = df['tokens'].apply(lambda x: ' '.join(x))

# Output a tfidf numpy matrix where each row is a document and each column is a word
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1,1))
X = vectorizer.fit_transform(df['joined_tokens'])

In [101]:
# Save all the features to binarized numpy files
with open('doc2vec.npy', 'wb') as f:
    array = np.stack(df['doc2vec'].values)
    np.save(f, array)

sp.sparse.save_npz('publications.npz', publications)

sp.sparse.save_npz('authors.npz', authors)

sp.sparse.save_npz('tfidf.npz', X)